### Extended Gloss Overlaps (Extended Lesk)


The algorithm measures the relatedness of two words. Just like Simple Lesk, it counts the overlaps of glosses, however it takes into account the related glosses of the two words as well.

Suppose we want to obtain the sense for a word in a certain context (for example a sentence or just a window of text). The steps of the algorithm are:

1. We first tag the words in the sentence with their part of speech
2. For each word we obtain the list of synsets corresponding to that part of speech.
3. For each synset we obtain the glosses of the synsets for all:
- hypernyms
- hyponyms
- meronyms
- holonyms
- troponyms
- attributes
- similar-tos
- also-sees

It is good to use a structure that shows for each gloss from which synset it comes from (in order to do the tests in the exercise). We add them all in a list with all the glosses (for each target word). We call these lists "extended glosses".

4. For each synset of the target word (i.e. the word for which we want to obtain the sense) we compute a score by counting the overlaps in the synset with all the other synsets corresponding to the words in the context. In computing the score, for each single word that appears in both extended glosses we add 1. However if it appears in a common phrase, supposing the length of common phrase is L, we add L^2 (for example, if "white bread" appears in both glosses, we add 4). We obviusly don't add the score for the separate words in the phrase. We try to find the longest common sequences of consecutive words (it shouldn't start or end with a pronoun, preposition, article or conjunction in both glosses). In order to avoid counting the same overlap multiple times for the same two glosses, after counting the overlap you should replace the sequence of words with a special string that doesn't appear in the text (don't remove it completely as you may obtain false overlaps). For example, you can use as special string "###".
5. After computing the score for each synset of the target word, choose as result the synset with the highest score

### Extended Lesk Exercise

Implement extended Lesk algorithm. Experiment with the measure by using different WordNet relations in computing the score. For a list of 7-10 synsets, print the measure for each pair of synsets (without repeating the synsets) with five different sets of at least three relations taken into acount in measuring the score. Write the observations. Just like in the exercise from the previous lab, try to obtain the word sense for the given text and word and print its definition. Can you find a text and word where simple Lesk gives the wrong answer, however extended Lesk gives the right answer?

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from nltk.corpus import stopwords
nltk.download('stopwords')


stopwords_ = stopwords.words('english')

synset_names = ['dog.n.01', 'cat.n.01', 'car.n.01', 'bicycle.n.01', 'fruit.n.01', 'banana.n.02', 'computer.n.01']
synsets = [wn.synset(name) for name in synset_names]

relation_sets = {
    'set1': ['hypernyms', 'hyponyms', 'member_meronyms'],
    'set2': ['hypernyms', 'attributes', 'similar_tos'],
    'set3': ['hypernyms', 'part_holonyms', 'member_holonyms'],
    'set4': ['hyponyms', 'substance_meronyms', 'verb_troponyms'],
    'set5': ['also_sees', 'attributes', 'member_meronyms']
}

def tokenize(text):
    return [t.lower() for t in nltk.tokenize.word_tokenize(text) if t.isalpha()]


def get_extended_gloss(syn, relations):
  glosses = [syn.definition()]
  glosses += syn.examples()
  for rel in relations:
    if hasattr(syn, rel):
      related = getattr(syn, rel)()
      for r in related:
        glosses.append(r.definition())
        glosses += r.examples()
  return glosses

g1 = get_extended_gloss(synsets[2], relation_sets['set1'])
g2 = get_extended_gloss(synsets[3], relation_sets['set1'])

def longest_common_sequence(a, b):
  dp = [[0] * (len(b) + 1) for _ in range(len(a) + 1)]
  max_len, end_i, end_j = 0, 0, 0
  for i in range(1, len(a) + 1):
    for j in range(1, len(b) + 1):
      if a[i - 1] == b[j - 1]:
        dp[i][j] = dp[i - 1][j - 1] + 1
        if dp[i][j] > max_len:
          max_len = dp[i][j]
          end_i, end_j = i, j
  return max_len, end_i, end_j

def overlap_score(gloss1, gloss2):
  g1 = tokenize(gloss1)
  g2 = tokenize(gloss2)
  score = 0
  while True:
    L, i, j = longest_common_sequence(g1, g2)
    if L <= 1:
      break
    phrase = g1[i - L:i]
    if phrase[0] not in stopwords_ and phrase[-1] not in stopwords_:
      score += L * L
    g1 = g1[:i - L] + ["###"] + g1[i:]
    g2 = g2[:j - L] + ["###"] + g2[j:]
  for w in set(g1) & set(g2):
    if w != "###" and w not in stopwords_:
      score += 1
  return score

def extended_overlap(s1, s2, relations):
  glosses1 = get_extended_gloss(s1, relations)
  glosses2 = get_extended_gloss(s2, relations)
  total = 0
  for g1 in glosses1:
    for g2 in glosses2:
      total += overlap_score(g1, g2)
  return total

for i in range(len(synsets)):
  for j in range(i+1, len(synsets)):
    s1, s2 = synsets[i], synsets[j]
    scores = {name: extended_overlap(s1, s2, rels) for name, rels in relation_sets.items()}
    print(f"{s1.name()} & {s2.name()}: {scores}")

context = "He went fishing for bass in the stream."
simple = lesk(nltk.tokenize.word_tokenize(context), 'bass', 'n')
print("\nSimple Lesk: ", simple, ": \n", simple.definition() if simple else None)

best_score = -1
best_syn = None
for s in wn.synsets('bass', 'n'):
  score = sum(extended_overlap(s, s2, relation_sets['set1'])
              for w in ['fishing', 'stream']
              for s2 in wn.synsets(w, 'n'))
  if score > best_score:
    best_score = score
    best_syn = s
print("\nExtended Lesk:", best_syn, ": \n", best_syn.definition())